In [1]:
import pandas as pd
import quandl

# Install the quandl package if it's not already installed
# !pip install quandl

# Set the API key
quandl.ApiConfig.api_key = "xyrEL2EBKf8XvGzp1YdA"

# Load the CSV file
robinhood_profit_df = pd.read_csv('Robinhood_Profit.csv')

# Extract the unique tickers from the CSV file
tickers = robinhood_profit_df['Instrument'].unique()

# Function to check if a ticker is valid
def is_valid_ticker(ticker):
    try:
        # Try fetching a single row of data to validate the ticker
        quandl.get(f"WIKI/{ticker}", rows=1)
        return True
    except Exception:
        return False

# Filter out invalid tickers
valid_tickers = [ticker for ticker in tickers if is_valid_ticker(ticker)]

# Prepare an empty DataFrame to store the data
data_frames = []

# Fetch the data for each valid ticker and store it in the DataFrame
for ticker in valid_tickers:
    try:
        # Fetch the current data for the ticker
        stock_data = quandl.get(f"WIKI/{ticker}", rows=1)
        stock_data['Ticker'] = ticker
        data_frames.append(stock_data)
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Concatenate all the individual DataFrames into one
if data_frames:
    data = pd.concat(data_frames, ignore_index=True)
    # Display the collected data
    print(data)
else:
    print("No valid data fetched.")

# Check if the data was collected and concatenate it into a DataFrame
if data_frames:
    data = pd.concat(data_frames, ignore_index=True)
    # Display the collected data
    print(data)
else:
    print("No valid data fetched.")


/Users/ardadinc/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


       Open      High      Low     Close     Volume  Ex-Dividend  Split Ratio  \
0    1.8000    1.8900    1.700    1.8492    72531.0          0.0          1.0   
1   10.6900   10.8600   10.451   10.5300   557983.0          0.0          1.0   
2  140.5000  140.5700  140.500  140.5200  1189653.0          0.0          1.0   
3    3.4672    3.4672    3.350    3.3900     1925.0          0.0          1.0   
4    3.0900    3.2000    2.970    3.1100  1222100.0          0.0          1.0   
5    2.3600    2.4390    2.220    2.2600  2135214.0          0.0          1.0   
6    2.8000    2.8300    2.670    2.7100   120485.0          0.0          1.0   

   Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume Ticker  
0     1.8000     1.8900     1.700      1.8492      72531.0   BPTH  
1    10.6900    10.8600    10.451     10.5300     557983.0   SNCR  
2   140.5000   140.5700   140.500    140.5200    1189653.0    PRE  
3     3.4672     3.4672     3.350      3.3900       1925.0   MIND  
4     3.090